In [1]:
import collections
from operator import itemgetter
import pandas as pd
import os 
import time
from tqdm import tqdm

In [19]:
import collections
from operator import itemgetter
import pandas as pd
import os 
import time
from tqdm import tqdm

class Graph_Coloring:
    df = None
    nb_nodes=0
    nb_edges=0
    
    def init_graph(self):
        result = []
        for i in range(self.nb_nodes):
            result.append({'status': 0,
                           'color': None,
                           'degree': 0,
                           'node': i,
                           'neighbors': [],
                           'dsat': 0
                           })
        return result
    
    def get_degrees(self, tab):
        for index, line in self.df.iterrows():
            tab[line['n1']-1]['degree'] += 1
            tab[line['n2']-1]['degree'] += 1
            tab[line['n1']-1]['neighbors'].append(line['n2']-1)
            tab[line['n2']-1]['neighbors'].append(line['n1']-1)
                   
        return tab
    
    def sort_by_degree(self, tab):
        tab = sorted(tab, key=itemgetter('degree'), reverse=True)
        new_tab = []
        for i, line in enumerate(tab):
            new_tab.append(line['node'])
        return new_tab
        
    def update_dsat(self, graph_number, tab):
        for i in tab[graph_number]['neighbors']:
            tab[i]['dsat'] += 1
            
    def choice(self, tab, sorted_tab):
        node = 0
        dsat_max = -1
        double = []
        for i, line in enumerate(tab):
            if (dsat_max <= line['dsat']) and (line['status'] == 0):
                if dsat_max == line['dsat']:
                    double.append(i)
                else:
                    dsat_max = line['dsat']
                    node = i
                    double = []
        
        if double != []:
            degree_max = tab[node]['degree']
            for i in double:
                if degree_max < tab[i]['degree']:
                    degree_max = tab[i]['degree']
                    node = i
        
        return node
        
    def choice_color(self, colors, node, tab):
        isUsed = False
        for color in colors:
            isUsed = False
            for i in tab[node]['neighbors']:
                if tab[i]['color'] == color:
                    isUsed = True
                    
            if isUsed:
                continue
            tab[node]['color'] = color
            tab[node]['status'] = 1
            return tab, False
        
        tab[node]['color'] = colors[len(colors) -1] +1
        tab[node]['status'] = 1
        return tab, True
    
    def color_check(self, tab):
        for i, line in enumerate(tab):
            for j in line['neighbors']:
                if line['color'] == tab[j]['color']:
                    return False
        return True
    def findStart(self, file_name):
        file = open(file_name, "r")
        i = 0
        found = False
        while(not found):
            nextLine = file.readline()
            i += 1
            if("p edge " in nextLine):
                splitted = nextLine.split()
                numberOfNodes = splitted[2]
                numberOfEdges = splitted[3]
                found = True
        file.close()
        return i, int(numberOfNodes), int(numberOfEdges)
    
    def get_df(self, file_name: str):
        ROWS_TO_SKIP, nb_nodes, nb_Edges = self.findStart(file_name)
        df = pd.read_csv(file_name, header=None, skiprows=ROWS_TO_SKIP, sep=' ', names=['e','n1','n2'])
        df = df.drop(['e'],axis=1)
        return df, nb_nodes, nb_Edges
    
    
    def recording_result(self, file_name, nb_colors, sol_file_name):
        to_write="{} | {} | {} | {}".format(file_name,self.nb_nodes, self.nb_edges, nb_colors)
        with open(sol_file_name, "w") as file:
            file.write(to_write)
    
    def start(self, file_name):
        print('Start')
        t_start = time.perf_counter()   
        print('Reading graph')
        self.df, self.nb_nodes, self.nb_edges = self.get_df(file_name)
        print('Running of graph coloring algorithm')
        result = self.init_graph()
        result = self.get_degrees(result)
        sorted_result = self.sort_by_degree(result)
        colors = [0]
        index = sorted_result[0]
        result[index]['color'] = 0
        result[index]['status'] = 1
        self.update_dsat(index, result)
        for i in tqdm(range(self.nb_nodes-1)):
            node = self.choice(result, sorted_result)
            result, new_color = self.choice_color(colors, node, result)
            if new_color:
                colors.append(colors[len(colors) -1] +1)
            self.update_dsat(node, result)
        check=self.color_check(result)
        print(check)
            
        print('\nNumber of colors:', len(colors))
        self.recording_result(file_name, len(colors), 'solution.sol')
        print('Total time:', int(time.perf_counter() - t_start), ' s')


Graph_Coloring().start('dsjc1000.5.col')


Start
Reading graph
Running of graph coloring algorithm


100%|██████████| 999/999 [00:07<00:00, 140.81it/s]


True

Number of colors: 125
Total time: 37  s
